In [ ]:
! pip install liac-arff

In [ ]:
## Checks to be executed for each dataset
## 1. The unique values across the entire dataset should be 0 or 1 only
## 2. There should NOT be any columns in the co-variate space with all value equal. Let's remove those columns

In [ ]:
import arff
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np

In [ ]:
path = "/repos/smote_msfb/public_datasets/tmc2007/"

In [ ]:
# Load the .arff file - With full data in one file
with open(path + 'tmc2007.arff', 'r') as f:
    dataset = arff.load(f)
    
# Extract data and attributes
data = dataset['data']
attributes = dataset['attributes']

# Create DataFrame
columns = [attr[0] for attr in attributes]
df = pd.DataFrame(data, columns=columns)

In [ ]:
print("Shape of the medical.arff file :", df.shape)

In [ ]:
### Get the position of the first Label columns
col_position = df.columns.get_loc("class01")  ## The first label column in the dataset
print(col_position)

In [ ]:
df.columns

In [ ]:
df.iloc[:,:col_position+1].columns

In [ ]:
##Check if the unique values across the entire co-variate space is 0 and 1 only or not
unique_values = np.unique(df.iloc[:, :col_position].values)
print(unique_values)

In [ ]:
## Check if the unique values in the mulit labels is 0 and 1 only or not
unique_values = np.unique(df.iloc[:,col_position:].values)
print(unique_values)

In [ ]:
## Check to drop columns from the dataset with same values for all rows

# Subset the first 1836 columns
features = df.iloc[:, :col_position]

# Identify columns with more than one unique value
non_constant_cols = features.loc[:, features.nunique(dropna=False) > 1]

print("Shape of the non_constant_cols :", non_constant_cols.shape)

# Concatenate with the remaining part of the DataFrame (e.g., label columns)
df_cleaned = pd.concat([non_constant_cols, df.iloc[:, col_position:]], axis=1)

print("Shape of df_cleaned :", df_cleaned.shape)

In [ ]:
## WE REMOVE A LOT OF COLUMNS WITH SAME VALUE ACROSS ALL ROWS. ALMOST 90% OF THE COLUMNS ARE GONE

col_position = non_constant_cols.shape[1]

col_position

In [ ]:
df_cleaned.iloc[:,:col_position+1].columns

In [ ]:
##  Rename the co-variate feature space of the dataset

# Generate new column names
new_feature_names = [f"f_{i}" for i in range(col_position)]

# Assign the new names to the first 1836 columns
df_cleaned.columns.values[:col_position] = new_feature_names

In [ ]:
df_cleaned.head(3)

In [ ]:
df_cleaned.drop('class02', axis=1, inplace=True)

In [ ]:
## Dataset is highly imbalanced

# Slice only the label columns
label_data = df_cleaned.iloc[:, col_position:]

# Calculate proportions of 0's and 1's for each column
for col in label_data.columns:
    value_counts = label_data[col].value_counts(normalize=True).sort_index()
    prop_0 = value_counts.get(0, 0)
    prop_1 = value_counts.get(1, 0)
    print(f"{col}: 0's = {prop_0:.4f}, 1's = {prop_1:.4f}")

In [ ]:
## Rename all the response variables as target_X

# Total number of columns
total_cols = df_cleaned.shape[1]

# Generate new names for response variables
num_targets = total_cols - col_position
new_target_names = [f"target_{i+1}" for i in range(num_targets)]

# Apply the new names
df_cleaned.columns.values[col_position:] = new_target_names

In [ ]:
df_cleaned.head(3)

In [ ]:
df_cleaned = df_cleaned.loc[:, (df_cleaned != 0).any(axis=0)]

In [ ]:
df_cleaned.shape

In [ ]:
df_cleaned.columns = [str(col) for col in df_cleaned.columns]

In [ ]:
# Save as Parquet (compressed with snappy)
df_cleaned.to_parquet(
    path + "/Final_dataset.parquet", 
    engine="pyarrow", 
    compression="snappy", 
    index=False
)

In [ ]:
df_cleaned.to_csv(path + "/Final_dataset.zip", index=False, compression='zip')